In [3]:
import cx_Oracle
import pandas as pd
import os
import pyarrow.parquet as pq

In [4]:
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="csebsd2db.cswg.com:1521/csebsd2_int"
)

In [7]:
item_master_select = "SELECT * FROM apps.hz_locations where rownum < 2"
item_master_df= pd.read_sql_query(item_master_select, source_conn)
item_master_df

,LOCATION_ID,LAST_UPDATE_DATE,LAST_UPDATED_BY,CREATION_DATE,CREATED_BY,LAST_UPDATE_LOGIN,REQUEST_ID,PROGRAM_APPLICATION_ID,PROGRAM_ID,PROGRAM_UPDATE_DATE,...,OBJECT_VERSION_NUMBER,CREATED_BY_MODULE,APPLICATION_ID,TIMEZONE_ID,GEOMETRY_STATUS_CODE,ACTUAL_CONTENT_SOURCE,VALIDATION_STATUS_CODE,DATE_VALIDATED,DO_NOT_VALIDATE_FLAG,GEOMETRY_SOURCE
0,9489,2021-09-08 10:50:45,7940,2013-10-20 14:46:36,3237,-1,None,None,None,None,...,7,HZ_CPUI,None,4,DIRTY,USER_ENTERED,None,None,None,None


In [5]:
# import pandas as pd
# import cx_Oracle

# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="csebsd2db.cswg.com:1521/csebsd2_int"
)
source_cursor = source_conn.cursor()

# Read the query from the file
with open('CM_full_cursor.sql', 'r') as file:
    query = file.read()

source_cursor.execute(query)
df = source_cursor.fetchall()

# Create a DataFrame from the fetched data
columns_info = [(desc[0], desc[1]) for desc in source_cursor.description]
columns = [desc[0] for desc in source_cursor.description]
cust_df = pd.DataFrame(df, columns=columns)

# Display the data
print("The DataFrame of GTT:")
print(cust_df)

# Access 'PARTY_SITE_ID' column and convert it to a list
party_site_ids = cust_df['PARTY_SITE_ID'].tolist()

# Display the list of party_site_ids
print("Party Site IDs:")
print(party_site_ids)

print('len(party_site_ids):', len(party_site_ids))

# Define the batch size (e.g., 1000)
batch_size = 1000

# Calculate the number of batches
num_batches = (len(party_site_ids) + batch_size - 1) // batch_size
print(num_batches)

# Assuming you have a database connection and a cursor named 'source_cursor'
# Adjust the SQL query based on your database and table structure
query_template = "SELECT * FROM xxfin.xxar_customer_master WHERE party_site_id IN ({})"

# Create an empty list to store DataFrames from each batch
all_dfs = []

# Iterate through batches
for i in range(num_batches):
    # Get the start and end indices for the current batch
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    # Extract the party_site_ids for the current batch
    current_batch = party_site_ids[start_idx:end_idx]

    # Generate the SQL query for the current batch
    current_query = query_template.format(','.join(map(str, current_batch)))

    # Execute the query and fetch the results
    source_cursor.execute(current_query)
    results = source_cursor.fetchall()

    # Create a DataFrame from the results
    current_columns = [desc[0] for desc in source_cursor.description]
    current_df = pd.DataFrame(results, columns=current_columns)

    # Append the current DataFrame to the list
    all_dfs.append(current_df)

# Concatenate all DataFrames into a single DataFrame
audit_df = pd.concat(all_dfs, ignore_index=True)

# Display the final DataFrame
print(audit_df)

# Get the number of rows in the final DataFrame
print(f"Number of rows in the final DataFrame: {audit_df.shape[0]}")


The DataFrame of GTT:
      PARTY_SITE_ID  STORE_NBR                 STORE_NAME STORE_STATUS  \
0             25275      20054  BLACKIE'S FARM FRESH PROD       Active   
1            376902      25317         FAIRWAY E-COMMERCE       Active   
2            376998      25401     IMPERIAL BAGS PAPER CO       Active   
3            376999      25403     N & P FOOD SERVICES IN       Active   
4            377000      25415     PLAINFIELD TOB & CANDY       Active   
...             ...        ...                        ...          ...   
2392        3467192      45018      JIB #8203- UMI STREET       Active   
2393        3470193  947230-MX      SEVEN UP CO DSD 13200       Active   
2394        3470194  947244-MX    FRITO LAY INC DSD 80024       Active   
2395        3470195  947291-MX      BERRES BROTHER COFFEE       Active   
2396        3470196  947886-MX             ANHEUSER BUSCH       Active   

     STORE_BILL_TO_FLAG STORE_SHIP_TO_FLAG  ACCT_ID ACCT_NBR  \
0                     Y  

      PARTY_SITE_ID  STORE_NBR                   STORE_NAME STORE_STATUS  \
0             25275      20054    BLACKIE'S FARM FRESH PROD       Active   
1            376902      25317           FAIRWAY E-COMMERCE       Active   
2            376998      25401       IMPERIAL BAGS PAPER CO       Active   
3            376999      25403       N & P FOOD SERVICES IN       Active   
4            377000      25415       PLAINFIELD TOB & CANDY       Active   
...             ...        ...                          ...          ...   
2387        2678359    9854083                FRED'S MARKET       Active   
2388        2859183      30115  CEDAR KNOLLS FARMERS MARKET       Active   
2389        3040128      30124                       C-TOWN       Active   
2390        3048123      30125                TUSCANY FOODS       Active   
2391        3460192  999999994               TEST 999999994       Active   

     STORE_BILL_TO_FLAG STORE_SHIP_TO_FLAG  ACCT_ID   ACCT_NBR  \
0                    

In [8]:
columns_info = [(desc[0], desc[1]) for desc in source_cursor.description]
columns_info

[('PARTY_SITE_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('STORE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_STATUS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_BILL_TO_FLAG', <cx_Oracle.DbType DB_TYPE_CHAR>),
 ('STORE_SHIP_TO_FLAG', <cx_Oracle.DbType DB_TYPE_CHAR>),
 ('ACCT_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ACCT_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_STATUS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_CUST_CLASS_CODE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_CREATION_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('CHAIN_NBR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('CHAIN_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CHAIN_SHORT_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CHAIN_INACTIVE_DATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('START_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STRT_CUST_NBR2', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('

In [1]:
print("Number of columns in columns variable:", len(columns_info))
print("Columns:", columns)

NameError: name 'columns_info' is not defined

In [7]:
import pandas as pd

# Assuming you have already created source_cursor and df

# Create columns_info
columns_info = [(desc[0], desc[1]) for desc in source_cursor.description]

# Create DataFrame
columns = [desc[0] for desc in source_cursor.description]
cust_df = pd.DataFrame(df, columns=columns)

# Function to format values based on data type
def format_value(value, data_type):
    if data_type.startswith('VARCHAR'):
        return f"'{value}'"
    elif data_type.startswith('DATE'):
        return f"TO_DATE('{value}', 'YYYY-MM-DD')"  # Adjust the date format as needed
    else:
        return str(value)

# Generate insert script for xxfin.xxar_customer_master_audit
insert_script = f"INSERT INTO xxfin.xxar_customer_master_audit ({', '.join(columns)}) VALUES\n"

for index, row in cust_df.iterrows():
    values = [format_value(row[col], data_type) for col, data_type in columns_info]
    values_str = ', '.join(values)
    insert_script += f"({values_str}),\n"

# Remove the trailing comma and newline
insert_script = insert_script.rstrip(',\n')

# Print or use the insert_script as needed
print(insert_script)



ValueError: 331 columns passed, passed data had 316 columns

In [ ]:
def generate_insert_query(table_name, column_names, bind_variables):
    # Start building the query
    query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ("

    # Add placeholders for bind variables
    query += ', '.join([f':{i + 1}' for i in range(len(bind_variables))])

    # Complete the query
    query += ")"

    # Return the final query
    return query


def generate_to_number_date_statements(column_names):
    statements = []
    for i, column in enumerate(column_names, start=1):
        if 'date' in column.lower():
            statements.append(f"TO_DATE(:{i}, 'YYYY-MM-DD HH24:MI:SS')")
        elif 'number' in column.lower():
            statements.append(f"TO_NUMBER(:{i})")
        else:
            statements.append(f":{i}")

    return statements


def generate_full_query(table_name, column_names):
    # Generate TO_NUMBER and TO_DATE statements
    conversion_statements = generate_to_number_date_statements(column_names)

    # Generate the insert query
    insert_query = generate_insert_query(table_name, column_names, conversion_statements)

    return insert_query


# Example usage
table_name = "xxfin.xxar_customer_master_audit"
column_names = 

full_query = generate_full_query(table_name, column_names)
print(full_query)


In [6]:
# Assuming `data_types_list` is your list of column names and types
data_types_list = [('PARTY_SITE_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('STORE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_STATUS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_BILL_TO_FLAG', <cx_Oracle.DbType DB_TYPE_CHAR>),
 ('STORE_SHIP_TO_FLAG', <cx_Oracle.DbType DB_TYPE_CHAR>),
 ('ACCT_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ACCT_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_STATUS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_CUST_CLASS_CODE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCT_CREATION_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('CHAIN_NBR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('CHAIN_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CHAIN_SHORT_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CHAIN_INACTIVE_DATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('START_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STRT_CUST_NBR2', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('END_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('END_CUST_NBR2', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PRIVATE_LABEL_OFFSET', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SUB_AUTH_OFFST', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AD_START_DAY_OF_WEEK', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('COMPANY_CODE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('HQ_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('DIVISION_NBR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('DIVISION_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('EBS_GL_CUST_GROUP_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('EBS_GL_CUST_GROUP_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_GROUP_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_GROUP_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GROUP_BUY_NBR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('STORE_CREATION_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('STORE_INACTIVE_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('STORE_LAST_UPDATE_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('LOCATION_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('STORE_ADDRESS1', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_ADDRESS2', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_ADDRESS3', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_ADDRESS4', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_CITY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_POSTAL_CODE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_STATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_PROVINCE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_COUNTY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_COUNTRY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_TELEPHONE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CUST_STORE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CUST_ACCT_PROFILE_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('PAY_TERMS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PAY_TERM_DESC', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PAY_TERM_START_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('PAY_TERM_END_DATE', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('COLLECTOR_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_CYCLE_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ALT_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BASE_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CUST_STATEMENT_FORMAT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PACA_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('TOBACCO_BILLABLE_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('TOBACCO_REBATE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ZONE_NBR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('MILITARY_DODAAC', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SALES_REPRESENTATIVE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BILL_TO_STORE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BILL_TO_STORE_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PRIMARY_CONTACT_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PRIMARY_CONTACT_PHONE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PRIMARY_CONTACT_EMAIL_ADDR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CUSTOMER_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('FINANCE_CHARGES', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('INTEREST_RATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PAYMENT_GRACE_DAYS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('INTEREST_PERIOD_DAYS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_ADDRESS1', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_CITY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_STATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_ZIP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('FINANCE_CHARGE_EFFECTIVE_DATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SUMMARY_STMT_FORMAT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('DELINQUENT_START_DATE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AR_SEGMENT_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AR_GROUP_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('IMPORTED_SOURCE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('REMITTANCE_ADDR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_TOLRNCE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('INVOICE_TOLRNCE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SALES_REP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RISK_TIER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_RETAIL_ACCTNG_REPORT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_LEVEL', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RELATED_CUST_LOCKBOX_APPLLY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SEND_ACH_NOTIFTN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STOCK_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('MULTI_STORE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('EFT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PICKUP_DAY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PAY_DUE_DAY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CREDIT_HOLD', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PAYMENT_TERMS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CREDIT_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BLANK_CHECK_CUSTOMER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AD_PLAN_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_PARTY_SITE_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_MULTI_STORE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PATRONAGE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PRINT_STATEMENTS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STATEMENT_INSTRUCTIONS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SEND_TO_AWI_CDH_FLG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_BILLING_CYCLE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_TERMS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_NP_XFER_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_PRINT_STMT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_RET_ACCT_STMT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_STMT_COPIES', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_GROUP_CODE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_PAYMENT_METHOD', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_STMT_LABELS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_PAYMENT_DUE_DAYS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_SUB_STORE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CM_CUST_XREF_NO', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STMT_ADJ_GROUP_CODE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PSUEDOEPHEDRINE_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CALL_FOR_CUSTOMER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CIGARETTE_LICENSE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SMALL_PACK', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CIGARETTE_VALIDATION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_CDH_CUSTOMER_CLASS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GAS_REWARDS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('MILES', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('PRINT_STMTS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_CDH_RACK_SERVICE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RETAIL_TYPE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('TAX_EXEMPT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ADV_FLAG_7', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ADV_FLAG6', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BB_AD_PLAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AD_COMMENTS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ADV_FLAG5', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CONFO_AD_PLAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('DEFAULT_TRIP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CATALOG10', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_PRINT_BULLETIN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_SIGN_KIT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SURVEYS', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ADV_FLAG8', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_AD_OFFSET', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('AGNE_CUSTOMER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CONFO', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('HBA_GM_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('HBA_GM_DISC_OVERIDE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('HBA_GM_DISC', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ITEM_ALLOWANCE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('COD', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SERV_CHARGE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('SINGLE_PICK', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AWI_CUSTOMER_TYPE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('MULTI_STORE_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('EIGHT_WEEK_BB', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BB_MOD_PROGRAM', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('KPC_END', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('KPC_RETAIL_ZONE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('KPC_START', <cx_Oracle.DbType DB_TYPE_DATE>),
 ('AWI_PRODUCT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SPECIALTY_FOODS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('D_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('D_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('D_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('D_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('D_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('D_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SY_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SY_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('SY_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SY_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SY_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SY_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('M_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('M_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('M_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('M_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('M_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('M_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('P_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('P_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('P_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('P_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('P_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('P_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GM_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GM_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GM_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GM_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GM_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GM_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('C_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('C_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('C_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('C_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('C_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('C_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('F_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('F_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('F_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('F_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('F_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('F_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RG_COST_PLUS_BRACKET_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RG_COST_PLUS_BRACKET_NUMBER', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('RG_MERCHANDISER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RG_PRICING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RG_SALESMAN', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RG_SKID_ROUNDING_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('DSD_VENDOR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('CUST_CODE_XREF', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SITE_PICK_SLIP_REQ', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SITE_PICK_SLIP_REQ_DISP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SITE_RANK', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACC_SUBS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('REJ_REASON', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('MAX_VOL_PALL', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ITEM_PAL_ROUND', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('LOG_ORD_QTY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('MIN_ORD_QTY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ENFORCE_DEPT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('MAX_ORD_QTY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PO_REQ', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PO_REQ_DISP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RET_PRC_REQ', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RET_PRC_REQ_DISP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CUST_XREF', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CUST_XREF_DISP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('LEGACY_CUST_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('LEGACY_COMPANY', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('BILL_TO_SITE_USE_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('SHIP_TO_SITE_USE_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('HOSTING_DEPT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('RETAILER_DEPT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('CONTACT_INFO', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_MANAGER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_OWNER', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_TYPE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('ACCEPT_SUBS', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('INT_PO_REQ', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('DIVISION_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('REGION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('DELEGATE_AUTRIZTN_FLG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('AISLE_LIMIT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('ACCT_TYPE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('OLD_STORE_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_0', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_0_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_1', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_1_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_2', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_2_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_3', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_3_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_4', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_4_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_5', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_5_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_6', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_6_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_7', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_7_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_8', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_8_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_9', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_DASH_9_DESCRIPTION', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('PLATINUM_DEPT', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('EXCLUDE_PDF_BKP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_ADV_CRD_GROC_RPK_CAYU_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_ADV_CRD_MEAT_BULK_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_ADV_CRD_CORDER_REPACK_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_ADV_CRD_FROZEN_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_ADV_CRD_BAKERY_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_ADV_CRD_REFRIG_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_ADV_CRD_ICREAM_PCT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_SALESMAN_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_SALESMAN_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_CM_DROP_SHIP_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_CM_PAYMENT_METHOD', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_SELECTION_SERVICE_FEE_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_FRZ_RBT_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_STMT_FROZ_ADJ_FACTOR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_STMT_MEAT_ADJ_FACTOR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_STMT_GROC_ADJ_FACTOR', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_NOTE_PAYOFF_CREDIT_RATE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_SALES_ADJ_REBATE_RATE', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_TRANSFER_BAL_DASH_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_CUST_CROSS_REF_DASH_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_REBATE_CHECK', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_GROUP_CODE_DESC', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_PORTAL_FOLDER_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('STORE_FAX_NBR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_AUD_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('CREDIT_LIMIT', <cx_Oracle.DbType DB_TYPE_NUMBER>),
 ('GS_MAP_CD', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_MAP_CD_LTR', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_VENDOR_PACK_IND', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_AD_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_TPR_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_GBY_GROUP', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_PRICING_ZONE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_SUPER_SERVE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_NABISCO', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_WILL_CALL', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('SWELL', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('HQ_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_PRIMARY_SHIP_FLAG', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_FUEL_ZONE', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_FUEL_ZONE_DESC', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_AD_GROUP_DESC', <cx_Oracle.DbType DB_TYPE_VARCHAR>),
 ('GS_PRICING_ZONE_DESC', <cx_Oracle.DbType DB_TYPE_VARCHAR>)]

# Create the INSERT INTO statement
insert_into = "INSERT INTO your_table_name ("

# Create lists to store bind variable names and column names
bind_variable_names = []
column_names = []

# Iterate through the data_types_list
for index, (col_name, col_type) in enumerate(data_types_list, start=1):
    # Append column names to the list
    column_names.append(col_name)

    # Append bind variable names to the list
    bind_variable_names.append(f":{index}")

# Combine the column names and bind variable names into the INSERT INTO statement
insert_into += ", ".join(column_names) + ") VALUES (" + ", ".join(bind_variable_names) + ")"

# Now you have the dynamically generated INSERT INTO statement
print(insert_into)


SyntaxError: invalid syntax (<ipython-input-6-45cc777d8d36>, line 2)